# Assignment 2 Visione Artificiale,          Rosario Bongiorno

___________________________________________________________________________________________________________________________________________________

Inizialmente importo i pacchetti e definisco le prime funzioni che serviranno

In [79]:

import os
import numpy as np
import cv2
import pickle
from sklearn.neighbors import KNeighborsClassifier as KNN
import re
import matplotlib.pyplot as plt


def ViolaJones(percorso):
    img = cv2.imread(percorso)
    faces_xml = 'haarcascade_frontalface_alt.xml'
    face_cascade = cv2.CascadeClassifier()
    # -- 1. Load the cascades
    if not face_cascade.load(faces_xml):
        print('--(!)Error loading face cascade')
        exit(0)
    
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_vj = face_cascade.detectMultiScale(img)
    volto=np.zeros(4096, dtype=np.float32)
    for (x, y, w, h) in img_vj:
        faceROI = img[y:y + h, x:x + w]
        volto=cv2.resize(faceROI, (64,64)).flatten()
 
    return volto

def project(gallery,mean,u):
    a=np.zeros((np.shape(gallery)[0],np.shape(u)[0]), dtype=np.float32)
    i=0
    for img in gallery:
        diff=img-mean
        a[i]=np.dot(diff,u.T)
        i=i+1
    return a

Procedo importando il dataset “All images aligned with funneling”, seleziono esclusvamente 2000 foto e ne calcolo la matrice media

In [82]:

c=0
face=np.zeros((2000,4096), dtype=np.float32)
sum=np.zeros((64,64), dtype=np.float32)
for cartella, sottocartelle, files in os.walk("./lfw_funneled"):
    for file in files:
        if c>=2000:
            break
        if file.endswith(".jpg"):
           
            img_vect = ViolaJones(cartella+"/"+file)
            face[c]=img_vect
            c=c+1
       


mean_vect=np.mean(face,axis=0)
mean=np.reshape(mean_vect, (64,64))
mean=mean.astype(np.uint8)

file = open('data/mean_vect', 'wb')
pickle.dump(mean_vect, file)
file.close()

cv2.imshow('img_media',mean)
cv2.waitKey(0)
cv2.destroyAllWindows()



Calcolo la matrice di covarianza, successivamente applico SVD, così da ottenere le gli autovettori che rappresentano i nostri egeinfaces

In [83]:
from sklearn.preprocessing import MinMaxScaler
for i in range(2000):
    face[i]=face[i]-mean_vect
    
C=np.cov(np.transpose(face))

#applico svd
u, s, vh = np.linalg.svd(C, full_matrices=True)
scaler = MinMaxScaler(feature_range=(0,255))
u = scaler.fit_transform(np.transpose(u))#eigenfaces

file = open('data/u', 'wb')
pickle.dump(u, file)
file.close()



Calcolo il numero di eigenfaces che mi serviranno per avere una varianza del 95% e del 99,99%

In [84]:

s=((s-s.min())/(s.max()-s.min()))
variance_sum = np.cumsum(s**2)
total_variance = np.sum(s**2)
cumulative_variance_ratio = variance_sum / total_variance


eig_95 = np.argmax(cumulative_variance_ratio >= 0.95) + 1
print("Numero di eigenfaces necessari per una varianza del 95%:", eig_95)
    
eig_99= np.argmax(cumulative_variance_ratio >= 0.9999) + 1
print("Numero di eigenfaces necessari per una varianza del 99,99%:", eig_99)    

Numero di eigenfaces necessari per una varianza del 95%: 4
Numero di eigenfaces necessari per una varianza del 99,99%: 140


Carico le immagini della gallery dei volti, applico Viola Jones e li salvo insieme alle rispettive etichette

In [85]:

i=0
gallery_img=np.zeros((48,4096), dtype=np.float32)
gallery_label=[]

for cartella, sottocartelle, files in os.walk("./gallery"):
    for file in files:
        
        if file.endswith(".png"):
            img_vect = ViolaJones(cartella+"/"+file)
            gallery_img[i]=img_vect
            gallery_label.append(re.sub('_[0-9]+.png', '', file))
            i=i+1
            
file = open('data/gallery_img', 'wb')
pickle.dump(gallery_img, file)
file.close()

file = open('data/gallery_label', 'wb')
pickle.dump(gallery_label, file)
file.close()

Estrapolo dal video effettuato in laboratorio i 20 frame che utilizzerò per il validation set.

NB. non eseguire questo codice altrimenti la label non corrisponderà con il validation set

In [10]:

cap = cv2.VideoCapture('./gallery/validation.mp4')
validation=[]
# Leggi i primi 20 frame del video
for i in range(60):
    
    if i%3==0:
        ret, test_img = cap.read()
        # Se la lettura del frame ha avuto successo
        if ret:
            test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

            faces_xml = 'haarcascade_frontalface_alt.xml'
            face_cascade = cv2.CascadeClassifier()
                # -- 1. Load the cascades
            if not face_cascade.load(faces_xml):
                    print('--(!)Error loading face cascade')
                    exit(0)

            faces = face_cascade.detectMultiScale(test_img)
            for (x, y, w, h) in faces:
                roi = test_img[y:y+h, x:x+w]
                test_feature = cv2.resize(roi, (64, 64)).flatten()
                validation.append(test_feature)
validationset=np.array(validation)


In [ ]:
file = open('data/validation', 'wb')
pickle.dump(validation, file)
file.close()

Creo La label del validation set

In [72]:
label_val=['antonio','rosario','n','daniele','alessandro','antonio','rosario','n','alessandro','daniele','antonio','rosario','n',
           'alessandro','daniele','antonio','rosario','n','alessandro','daniele','antonio','rosario','n','alessandro','daniele','rosario','antonio',
           'alessandro','daniele','antonio','rosario','daniele','alessandro','antonio','rosario','alessandro','daniele','n','antonio',
           'rosario','alessandro','daniele','n','antonio','rosario','daniele','alessandro','antonio','rosario','alessandro','daniele','antonio',
           'daniele','alessandro','antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','daniele','alessandro',
           'antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','daniele','alessandro','antonio','daniele','alessandro',
           'antonio','alessandro','daniele']
label_val=np.array(label_val)

file = open('data/label_val', 'wb')
pickle.dump(label_val, file)
file.close()
            

Per una visualizzazione più immediata dei dati eseguire il sequente blocco di codice

In [86]:
file = open('data/validation', 'rb')                           
validation= pickle.load(file)              
file.close()

file = open('data/label_val', 'rb')                           
label_val= pickle.load(file)              
file.close()

file = open('data/gallery_img', 'rb')                           
gallery_img= pickle.load(file)              
file.close()

file = open('data/gallery_label', 'rb')                           
gallery_label= pickle.load(file)              
file.close()

file = open('data/u', 'rb')                           
u= pickle.load(file)              
file.close()

file = open('data/mean_vect', 'rb')                           
mean_vect= pickle.load(file)              
file.close()


validation=np.delete(validation,[2,7,12,17,22,37,42],0)#elimino le immagini che non rappresentano dei volti
label_val=np.delete(label_val,[2,7,12,17,22,37,42])

        

Creo un classificatore KNN che addestro con il training set derivato dalla gallery

In [87]:

for num_eig in [5,140]:
    for k in [1,3,5]:
        
        #proietto i dati della gallery nello spazio delle eigenfaces
        a_training=project(gallery_img,mean_vect,u[:num_eig,])
        #proietto i dati del validation set nello spazio delle eigenfaces
        a_validation=project(validation,mean_vect,u[:num_eig,])
        
        
        # Crea il classificatore KNN
        knn = KNN(n_neighbors=k)
        #addestro il classificatore con il training set e le rispettive label
        knn.fit(a_training, gallery_label)
        #predico i risultati
        label = knn.predict(a_validation)
        #tamite il validation set ottengo una stima del mio classificatore
        accuratezza=knn.score(a_validation,label_val)
        if num_eig==5:
            var="95%"
        else:
            var="99,99%"
        print("Accuratezza con varianza:"+var+" e k:"+str(k)+"---------------->"+str(accuratezza))

Accuratezza con varianza:95% e k:1---------------->0.28169014084507044
Accuratezza con varianza:95% e k:3---------------->0.28169014084507044
Accuratezza con varianza:95% e k:5---------------->0.28169014084507044
Accuratezza con varianza:99,99% e k:1---------------->0.30985915492957744
Accuratezza con varianza:99,99% e k:3---------------->0.28169014084507044
Accuratezza con varianza:99,99% e k:5---------------->0.28169014084507044


Dai risultati ottenuti, la miglior combinazione è varianza al 99,99% e k=1, quindi effettuo il test

In [66]:

a_training=project(gallery_img,mean_vect,u[:140,])
a_validation=project(validation,mean_vect,u[:140,])
        
        
        # Crea il classificatore KNN
knn = KNN(n_neighbors=1)
knn.fit(a_training, gallery_label)



face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

# Apri il video
cap = cv2.VideoCapture('./gallery/test_1.mp4')

# Loop attraverso ogni frame del video
while True:
    # Leggi un frame dal video
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Applica l'algoritmo di Viola-Jones per rilevare le facce nel frame
    faces = face_cascade.detectMultiScale(gray)
    for (x, y, w, h) in faces:
        faceROI = gray[y:y + h, x:x + w]
        test_feature=cv2.resize(faceROI, (64,64)).flatten()
        test_feature=project(np.array([test_feature]),mean_vect,u[:140,])
        label = knn.predict(test_feature)
        cv2.putText(frame, f"{label}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        

    cv2.imshow('Video', frame)

    # Se viene premuto il tasto 'q' interrompi il loop
    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()